In [1]:
from src.model import GPTQModel
model = GPTQModel(model_name="anon8231489123/vicuna-13b-GPTQ-4bit-128g", device="cuda", wbits=4, groupsize=128)

/home/fred/mambaforge/envs/vicuna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model ...


/home/fred/mambaforge/envs/vicuna/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/fred/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/fred/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be removed in 

Done.


In [34]:
from src.conversation import Conversation, SeparatorStyle


default_conv = Conversation(
    system="A chat between a curious human and an artificial intelligence assistant. "
           "The assistant gives helpful, detailed, and polite answers to the human's questions.",
    roles=["Human", "Assistant"],
    messages=[
        ["Human", "Give three tips for staying healthy."],
        ["Assistant",
            "Sure, here are three tips for staying healthy:\n"
            "1. Exercise regularly: Regular physical activity can help improve your overall health and wellbeing. "
            "It can also help reduce your risk of chronic conditions such as obesity, diabetes, heart disease, "
            "and certain cancers. Aim for at least 150 minutes of moderate-intensity aerobic exercise or "
            "75 minutes of vigorous-intensity aerobic exercise per week, along with muscle-strengthening "
            "activities at least two days per week.\n"
            "2. Eat a balanced diet: Eating a balanced diet that is rich in fruits, "
            "vegetables, whole grains, lean proteins, and healthy fats can help support "
            "your overall health. Try to limit your intake of processed and high-sugar foods, "
            "and aim to drink plenty of water throughout the day.\n"
            "3. Get enough sleep: Getting enough quality sleep is essential for your physical "
            "and mental health. Adults should aim for seven to nine hours of sleep per night. "
            "Establish a regular sleep schedule and try to create a relaxing bedtime routine to "
            "help improve the quality of your sleep."]
    ],
    offset=2,
    sep_style=SeparatorStyle.SINGLE,
    sep="###",
)



result = model.inference(prompt="How black holes work, detailed explanation please.", conversation=default_conv)
result.last_message

"Black holes are extremely dense and massive objects in space that have such a strong gravitational pull that nothing, not even light, can escape once it gets too close to the event horizon.\nThe formation of black holes occurs when a massive star runs out of fuel and collapses under its own gravity. This collapse causes the star to become incredibly dense and hot, and the material inside the star is compressed to a point where it becomes a singularity, or a point of infinite density.\nThe event horizon of a black hole is the boundary beyond which nothing can escape, not even light. Once an object crosses the event horizon, it is trapped by the black hole's gravitational pull and cannot escape.\nBlack holes are classified based on their mass, with the most massive ones being supermassive black holes that are millions or billions of times more massive than the Sun. There are also smaller black holes that are only a few times more massive than the Sun, which are called stellar-mass black

In [21]:
from langchain.llms.base import LLM
from typing import Optional, List

class GPTQModelLLM(LLM):
    model: GPTQModel
    params: dict = {}
 
    @property
    def _llm_type(self) -> str:
        return "gptq_model_llm"

    def _call(self, prompt:str, stop: Optional[List[str]] = None) -> str:
        conv = model.inference(prompt=prompt, **self.params)
        if isinstance(conv, str):
            return conv
        return "Error"

In [24]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
prompt = PromptTemplate(
    input_variables=["human", "assistant", "question"],
    template="A chat between a curious {human} and an artificial intelligence {assistant}. "
           "The assistant gives helpful, detailed, and polite answers to the {human}'s questions."
           "Example interaction: ###{human}: What is the meaning of life? ###{assistant}: The meaning of life is to give life meaning."
           "###{human}: {question}")

In [32]:
llm = GPTQModelLLM(model=model, params={"temperature": 1.0})
llm(prompt.format(human="Marco", assistant="GPTQ", question="How black holes work, detailed explanation please."))

''